# Generate annual composite image

This step uses the Google Earth Engine (GEE) Python api to generate annual composite images of the study region defined by a Polygon or LineString. If using a LineString representing the coastline the coastal zone is generated using a 1500m buffer around the shoreline. If a Polygon, image will match the extent of the region defined by the Polygon. 

Outputs will be downloaded to the user's Google drive. This step will require the user to have a GEE account and to have the Python api authenticated (https://developers.google.com/earth-engine/guides/python_install). See [README](README.md) for more information.     

### Step 1 - generate and download composites

composite creation supports Sentinel-2, Landsat 7 and Landsat 8 multispectral sensors and Sentinel-1 SAR. 

| Sensor | Argument |
| :- | :-|
| Sentinel-2 | 'S2' |
| Sentinel-1 | 'S1' |
| Landsat 7 | 'LS7' |
| Landsat 8 | 'LS8' |

Run the following code block to generate SAR and optical composites for the year 2019 for the Auckland region using shapefile in the ```'inputs/examples/'``` folder.

In [ ]:
# import modules
from coastal_landcover_classification import composite
import os
import tqdm
import glob
import rsgislib
import rsgislib.imageutils 

In [ ]:
# define arguments 
year = 2019
region = 'inputs/example/auk-coastline.shp'
sensor = 'S2'
crs = 'EPSG:2193'
pixel_size = 20
down_folder = 'Auckland'

# create optical composite image 
composite.create_optical_composite(year, region, sensor, crs, pixel_size, use_toa=True, down_folder=down_folder)
 
# create SAR composite image
sensor = 'S1'
composite.create_sar_composite(year, region, sensor, crs, pixel_size, down_folder=down_folder)



The output images will be saved to google drive. Download and move them to the ```'inputs/examples/'``` folder before moving onto step 2. 

### Step 2 - replace NAN values with integer and mosaic composites with multiple tiles



In [ ]:
# iterate over S1 image
for img in tqdm.tqdm(glob.glob('inputs/example/S1*')):
    composite.set_nodata_val(img, -99)

# create img_list and mosaic S2 tiles
img_list = []
for img in glob.glob('inputs/example/S2*'):
    img_list.append(img)
print(img_list)

# define output_image 
output_img = 'inputs/example/S2-composite.kea'
composite.mosaic(img_list, output_img, nodataVal=-99, outputFormat='KEA', dataType=rsgislib.TYPE_32FLOAT)

### Step 3 - Mask composites to study area (urban areas and elevation masked) 

The final step before classification is masking composites images to the area of interest. In this example urban areas have been masked with ancillary data and elevation over 10m is also excluded.

GEE imagery is downloaded using the GeoTiff file format. The Kea file format is more efficient and uses less memory and is used in the classification workflow. The kealib library is included in the conda environment.  Users can output files as GTiff by changing the ```out_format``` argument to GTiff and filepath ends to ```'.tif'```.

In [ ]:
# define composite images
S1 = 'inputs/example/S1-2019.tif'
S2 = 'inputs/example/S2-composite.kea'
aoi = 'inputs/example/study-area-nztm.kea'
out_format = 'KEA'

# mask composite images
for i in S1, S2:
    outImg = i[:-4] + '-aoi.kea'
    band_names = rsgislib.imageutils.get_band_names(i)
    rsgislib.imageutils.mask_img(i, aoi, outImg, out_format, rsgislib.TYPE_32FLOAT, -99, 0)
    rsgislib.imageutils.set_band_names(outImg, band_names)
    composite.set_nodata_val(outImg, -99)

The masked composite images are now ready to used in the classification workflow. 